In [1]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
import LE4PD3N
import subprocess
import time
import os
import numpy.random as random

In [2]:
def traj_from_xtc(XTC, TOP, chunk_size = 5000):
	import mdtraj as md
	import numpy as np
	lst = []
	for chunk in md.iterload(XTC, chunk = chunk_size, top = TOP):
		print(chunk)
		lst.append(chunk)

	for i, dummy in enumerate(lst):
		print(dummy)
		NFRS = dummy.n_frames
		NATOMS = dummy.n_atoms
		if i == 0:
			traj =  np.reshape(dummy.xyz, (NFRS, 3*NATOMS))
		else:
			traj = np.vstack([traj, np.reshape(dummy.xyz, (NFRS, 3*NATOMS))])

	return traj.T

In [5]:
def tau_convert(eigvals, avfr, T = 300, bar = None, HA = False):
    #kT in J
    avfr = avfr*1e-9
    kT = 1.38e-23*T
    #Convert to kg *nm^2 / ps^2
    sigma = (kT / avfr)
    #print(sigma)
    tau = 1e-6*eigvals*avfr /(kT)
    if bar == None:
        bar = np.zeros(len(eigvals))
    tau_scaled = tau*np.exp(bar)
    return tau

In [6]:
# use values from universality paper

# scaling laws within error
a = 2.
gamma = 1.

# energy per unit length in kT / A
eps = (6.5 / 10.) / (1.38 * 0.3 * 6.022 *(1 / 4.184))

# From here on is the template for making the pickle based on the numpy files created by LE4PD-XYZ

In [7]:
TCF_final_1ebw = np.load('3ttp_all_tcfs.npy')
t_axis=TCF_final_1ebw[:,0]

HA_tcf_calc={}

In [8]:
t_axis

array([0.00000e+00, 5.00000e+00, 1.00000e+01, ..., 2.49990e+05,
       2.49995e+05, 2.50000e+05], shape=(50001,))

In [9]:
#HA
eigvals_list = []
eigvecs_list = []
C_eigvals_list = []
tau_list = []
tau_scaled_list = []
corr_func_list = []
corr_func_scaled_list = []
cdf_list = []
cdf_scaled_list = []
covar_list = []
dtraj_list = []

msas=[8,16,32,64,128,256,'full']
num_samples_per_msa=320
path_prefix='../'

for system_name in ['3ttp', '2pc0', '1q9p']:
    eigvals_list = []
    eigvecs_list = []
    C_eigvals_list = []
    tau_list = []
    tau_scaled_list = []
    corr_func_list = []
    corr_func_scaled_list = []
    cdf_list = []
    cdf_scaled_list = []
    covar_list = []
    dtraj_list = []
    SYS=system_name

    HA_tcf_calc[SYS]={}   #can iterate over sytem names

    Hs=[]
    covars=[]
    AIHIs=[]
    for i,msa in enumerate(msas):
        start_time_all = time.time()
        
        path_structures=path_prefix + f'{system_name}_{msa}msa/'
        os.chdir(path_structures)
        print(os.getcwd())
        XTC=path_structures + f'aligned.xtc'
        TOP=f"../{system_name}_CA.pdb"
        T=300
        NFRS = 320
        
        traj = traj_from_xtc(XTC, TOP)
        dtraj = traj - traj.mean(0)[None,:]
        QINV = np.load(path_structures + 'QINVmatrix.npy')
        Q = np.load(path_structures + 'Qmatrix.npy')
        eigvecs = np.copy(Q)
        eigvals = np.loadtxt(path_structures + 'lambda_eig')
        mu_eig = np.loadtxt(path_structures + 'mu_eig')
        eigvals_list.append(eigvals)
        eigvecs_list.append(Q)
        C_eigvals_list.append(mu_eig)
        
        fratio, sigma, fric, avfr = LE4PD3N.fric_calc(TOP, SYS, dtraj.shape[0] // 3, dtraj.shape[1], eigvals, 300, path_to_resarea=path_structures)
        tau = tau_convert(eigvals, avfr)
        tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))
        tau_list.append(tau)
        tau_scaled_list.append(tau_scaled)
        weights = np.zeros((eigvecs.shape[0] // 3, eigvecs.shape[1]))
        for n, _ in enumerate(range(0, eigvecs.shape[0], 3)):
            for a in range(eigvecs.shape[1]):
                #print(a, counter)
                weights[n,a] = (eigvecs[_:_+3,a]**2).sum(0)
        #t = np.linspace(0, 100000, 100000)
        import copy
        t = copy.deepcopy(t_axis) # known time axis values with 5ps gap
        corr_func = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau[None,:-6]))).T).T 
        corr_func_scaled = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau_scaled[None,:-6]))).T).T
        corr_func = corr_func / corr_func[0,:]
        corr_func_scaled = corr_func_scaled / corr_func_scaled[0,:]
        
        corr_func_list.append(corr_func)
        corr_func_scaled_list.append(corr_func_scaled)
        cdf_list.append(1. - corr_func)
        cdf_scaled_list.append(1. - corr_func_scaled)
        
        stop_time_all = time.time()
        
        print(f"Time taken for {msa}MSA is {(stop_time_all-start_time_all)/60:2.4}mins")

    import copy

    corr_func_list_ha=[]
    corr_func_scaled_list_ha=[]
    corr_func_list_ha=copy.deepcopy(corr_func_list)
    corr_func_scaled_list_ha=copy.deepcopy(corr_func_scaled_list)
    cdf_list=[]
    cdf_scaled_list=[]
    for n, corr_func in enumerate(corr_func_list):
        cdf_list.append(1-corr_func)
        cdf_scaled_list.append(1-corr_func_scaled_list[n])

    cdf_list_ha=copy.deepcopy(cdf_list)
    cdf_scaled_list_ha=copy.deepcopy(cdf_scaled_list)

    HA_tcf_calc[SYS]['AF2']={}
    HA_tcf_calc[SYS]['AF2']['non_scaled']={}  #for two different barrier treatment
    HA_tcf_calc[SYS]['AF2']['scaled']={}
    HA_tcf_calc[SYS]['AF2']['CinvH']={}    #this is Qmatrix.npy file but it is actually (CH^-1) = (HC^-1)^-1 = (HA)^-1 (should've named it properly I'm stoopid lol)
    HA_tcf_calc[SYS]['AF2']['C']={}
    HA_tcf_calc[SYS]['AF2']['non_scaled']['tcf']=corr_func_list_ha
    HA_tcf_calc[SYS]['AF2']['non_scaled']['cdf']=cdf_list_ha
    HA_tcf_calc[SYS]['AF2']['scaled']['tcf']=corr_func_scaled_list_ha
    HA_tcf_calc[SYS]['AF2']['scaled']['cdf']=cdf_scaled_list_ha
    HA_tcf_calc[SYS]['AF2']['CinvH']['eigvec']=eigvecs_list
    HA_tcf_calc[SYS]['AF2']['CinvH']['eigval']=eigvals_list  #lambda eig vals coming from the HC^-1 
    HA_tcf_calc[SYS]['AF2']['C']['eigval']=C_eigvals_list    #mu eig values from Covariance matrix
    HA_tcf_calc[SYS]['AF2']['msas']=msas
    HA_tcf_calc[SYS]['AF2']['n_samples']=num_samples_per_msa
    HA_tcf_calc[SYS]['AF2']['t_axis']=t


    BD = '/media/ebeyerle/seagate/af2-dynamics/notebooks'
    os.chdir(BD)
    HA_tcf_calc[SYS]['test'] = {}

    #each files contain tcfs and the time axis => shape = (frames,N_res+1) and the time axis is in tcf[:,0]
    #hence the cdf is offset by 1 index

    md_files = {'1000ns':f'{SYS}_all_tcfs.npy',
                '100ns':f'./100ns_{SYS}_all_tcfs.npy',
                '10ns':f'./10ns_{SYS}_all_tcfs.npy'}

    for i,timescale in enumerate(md_files.keys()):
        HA_tcf_calc[SYS]['test'][timescale] = {}
        HA_tcf_calc[SYS]['test'][timescale]['tcf'] = np.load(md_files[timescale])
        HA_tcf_calc[SYS]['test'][timescale]['cdf'] = 1 - HA_tcf_calc[SYS]['test'][timescale]['tcf'][:,1:]

    print(HA_tcf_calc.keys(),HA_tcf_calc[SYS].keys(),HA_tcf_calc[SYS]['AF2'].keys())
        

/media/ebeyerle/seagate/af2-dynamics/3ttp_8msa
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
3ttp 99 320
fratio:  0.20351838934321723
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 8MSA is 0.0223mins
/media/ebeyerle/seagate/af2-dynamics/3ttp_16msa
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
3ttp 99 320
fratio:  0.20274016712014448
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 16MSA is 0.02287mins
/media/ebeyerle/seagate/af2-dynamics/3ttp_32msa
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
3ttp 99 320
fratio:  0.20403490170448912
Temperature 

/tmp/ipykernel_2600544/2350508023.py:61: RuntimeWarning: invalid value encountered in sqrt
  tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))


Time taken for 8MSA is 0.01306mins
/media/ebeyerle/seagate/af2-dynamics/2pc0_16msa
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
2pc0 99 320
fratio:  0.20132278875535353
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 16MSA is 0.02381mins
/media/ebeyerle/seagate/af2-dynamics/2pc0_32msa
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
2pc0 99 320
fratio:  0.20159325671528877
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 32MSA is 0.02429mins
/media/ebeyerle/seagate/af2-dynamics/2pc0_64msa
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
2pc0 99 320
fra

In [10]:
suffix_list = ['_0eafb', '_5d034', '_87f3d']
for i,system_name in enumerate(['3ttp', '2pc0', '1q9p']):
    eigvals_list = []
    eigvecs_list = []
    C_eigvals_list = []
    tau_list = []
    tau_scaled_list = []
    corr_func_list = []
    corr_func_scaled_list = []
    cdf_list = []
    cdf_scaled_list = []
    covar_list = []
    dtraj_list = []
    SYS=system_name
    suffix = suffix_list[i]

    Hs=[]
    covars=[]
    AIHIs=[]
    start_time_all = time.time()
    
    path_structures=path_prefix + f'AF_Cluster-main/{system_name}{suffix}/'
    #os.chdir(path_structures)
    print(os.getcwd())
    XTC=path_structures + f'aligned.xtc'
    TOP=f"../{system_name}_CA.pdb"
    T=300
    NFRS = 320
    
    traj = traj_from_xtc(XTC, TOP)
    dtraj = traj - traj.mean(0)[None,:]
    QINV = np.load(path_structures + 'LE4PD/QINVmatrix.npy')
    Q = np.load(path_structures + 'LE4PD/Qmatrix.npy')
    eigvecs = np.copy(Q)
    eigvals = np.loadtxt(path_structures + 'LE4PD/lambda_eig')
    mu_eig = np.loadtxt(path_structures + 'LE4PD/mu_eig')
    eigvals_list.append(eigvals)
    eigvecs_list.append(Q)
    C_eigvals_list.append(mu_eig)
    
    fratio, sigma, fric, avfr = LE4PD3N.fric_calc(TOP, SYS, dtraj.shape[0] // 3, dtraj.shape[1], eigvals, 300, path_to_resarea=path_structures)
    tau = tau_convert(eigvals, avfr)
    tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))
    tau_list.append(tau)
    tau_scaled_list.append(tau_scaled)
    weights = np.zeros((eigvecs.shape[0] // 3, eigvecs.shape[1]))
    for n, _ in enumerate(range(0, eigvecs.shape[0], 3)):
        for a in range(eigvecs.shape[1]):
            #print(a, counter)
            weights[n,a] = (eigvecs[_:_+3,a]**2).sum(0)
    #t = np.linspace(0, 100000, 100000)
    import copy
    t = copy.deepcopy(t_axis) # known time axis values with 5ps gap
    corr_func = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau[None,:-6]))).T).T 
    corr_func_scaled = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau_scaled[None,:-6]))).T).T
    corr_func = corr_func / corr_func[0,:]
    corr_func_scaled = corr_func_scaled / corr_func_scaled[0,:]
    
    corr_func_list.append(corr_func)
    corr_func_scaled_list.append(corr_func_scaled)
    cdf_list.append(1. - corr_func)
    cdf_scaled_list.append(1. - corr_func_scaled)
    
    stop_time_all = time.time()
    
    print(f"Time taken for {system_name}{suffix} is {(stop_time_all-start_time_all)/60:2.4}mins")

    import copy
    corr_func_list_ha=[]
    corr_func_scaled_list_ha=[]
    corr_func_list_ha=copy.deepcopy(corr_func_list)
    corr_func_scaled_list_ha=copy.deepcopy(corr_func_scaled_list)
    cdf_list=[]
    cdf_scaled_list=[]
    for n, corr_func in enumerate(corr_func_list):
        cdf_list.append(1-corr_func)
        cdf_scaled_list.append(1-corr_func_scaled_list[n])

    cdf_list_ha=copy.deepcopy(cdf_list)
    cdf_scaled_list_ha=copy.deepcopy(cdf_scaled_list)

    HA_tcf_calc[SYS]['AFc']={}
    HA_tcf_calc[SYS]['AFc']['non_scaled']={}  #for two different barrier treatment
    HA_tcf_calc[SYS]['AFc']['scaled']={}
    HA_tcf_calc[SYS]['AFc']['CinvH']={}    #this is Qmatrix.npy file but it is actually (CH^-1) = (HC^-1)^-1 = (HA)^-1 (should've named it properly I'm stoopid lol)
    HA_tcf_calc[SYS]['AFc']['C']={}
    HA_tcf_calc[SYS]['AFc']['non_scaled']['tcf']=corr_func_list_ha
    HA_tcf_calc[SYS]['AFc']['non_scaled']['cdf']=cdf_list_ha
    HA_tcf_calc[SYS]['AFc']['scaled']['tcf']=corr_func_scaled_list_ha
    HA_tcf_calc[SYS]['AFc']['scaled']['cdf']=cdf_scaled_list_ha
    HA_tcf_calc[SYS]['AFc']['CinvH']['eigvec']=eigvecs_list
    HA_tcf_calc[SYS]['AFc']['CinvH']['eigval']=eigvals_list  #lambda eig vals coming from the HC^-1 
    HA_tcf_calc[SYS]['AFc']['C']['eigval']=C_eigvals_list    #mu eig values from Covariance matrix
    HA_tcf_calc[SYS]['AFc']['n_samples']=num_samples_per_msa
    HA_tcf_calc[SYS]['AFc']['t_axis']=t

/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 849 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 849 frames, 99 atoms, 99 residues, without unitcells>
3ttp 99 849
fratio:  0.20111609925391974
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 3ttp_0eafb is 0.01018mins
/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 1180 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 1180 frames, 99 atoms, 99 residues, without unitcells>
2pc0 99 1180
fratio:  0.2031466528355126
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0


/tmp/ipykernel_2600544/2224345573.py:43: RuntimeWarning: invalid value encountered in sqrt
  tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))


Time taken for 2pc0_5d034 is 0.01381mins
/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 623 frames, 95 atoms, 95 residues, without unitcells>
<mdtraj.Trajectory with 623 frames, 95 atoms, 95 residues, without unitcells>
1q9p 95 623
fratio:  0.20384718294951396
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 1q9p_87f3d is 0.01355mins


In [11]:
suffix_list = ['_0eafb', '_5d034', '_87f3d']
for i,system_name in enumerate(['3ttp', '2pc0', '1q9p']):
    eigvals_list = []
    eigvecs_list = []
    C_eigvals_list = []
    tau_list = []
    tau_scaled_list = []
    corr_func_list = []
    corr_func_scaled_list = []
    cdf_list = []
    cdf_scaled_list = []
    covar_list = []
    dtraj_list = []
    SYS=system_name
    suffix = suffix_list[i]

    Hs=[]
    covars=[]
    AIHIs=[]
    start_time_all = time.time()
    
    path_structures=path_prefix + f'microsoft-Graphormer-5e62370/distributional_graphormer/protein/{system_name}_output/'
    #os.chdir(path_structures)
    print(os.getcwd())
    XTC=path_structures + f'aligned.xtc'
    TOP=f"../{system_name}_CA.pdb"
    T=300
    NFRS = 320
    
    traj = traj_from_xtc(XTC, TOP)
    dtraj = traj - traj.mean(0)[None,:]
    QINV = np.load(path_structures + 'LE4PD/QINVmatrix.npy')
    Q = np.load(path_structures + 'LE4PD/Qmatrix.npy')
    eigvecs = np.copy(Q)
    eigvals = np.loadtxt(path_structures + 'LE4PD/lambda_eig')
    mu_eig = np.loadtxt(path_structures + 'LE4PD/mu_eig')
    eigvals_list.append(eigvals)
    eigvecs_list.append(Q)
    C_eigvals_list.append(mu_eig)
    
    fratio, sigma, fric, avfr = LE4PD3N.fric_calc(TOP, SYS, dtraj.shape[0] // 3, dtraj.shape[1], eigvals, 300, path_to_resarea=path_structures)
    tau = tau_convert(eigvals, avfr)
    tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))
    tau_list.append(tau)
    tau_scaled_list.append(tau_scaled)
    weights = np.zeros((eigvecs.shape[0] // 3, eigvecs.shape[1]))
    for n, _ in enumerate(range(0, eigvecs.shape[0], 3)):
        for a in range(eigvecs.shape[1]):
            #print(a, counter)
            weights[n,a] = (eigvecs[_:_+3,a]**2).sum(0)
    #t = np.linspace(0, 100000, 100000)
    import copy
    t = copy.deepcopy(t_axis) # known time axis values with 5ps gap
    corr_func = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau[None,:-6]))).T).T 
    corr_func_scaled = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau_scaled[None,:-6]))).T).T
    corr_func = corr_func / corr_func[0,:]
    corr_func_scaled = corr_func_scaled / corr_func_scaled[0,:]
    
    corr_func_list.append(corr_func)
    corr_func_scaled_list.append(corr_func_scaled)
    cdf_list.append(1. - corr_func)
    cdf_scaled_list.append(1. - corr_func_scaled)
    
    stop_time_all = time.time()
    
    print(f"Time taken for {system_name}{suffix} is {(stop_time_all-start_time_all)/60:2.4}mins")

    import copy
    corr_func_list_ha=[]
    corr_func_scaled_list_ha=[]
    corr_func_list_ha=copy.deepcopy(corr_func_list)
    corr_func_scaled_list_ha=copy.deepcopy(corr_func_scaled_list)
    cdf_list=[]
    cdf_scaled_list=[]
    for n, corr_func in enumerate(corr_func_list):
        cdf_list.append(1-corr_func)
        cdf_scaled_list.append(1-corr_func_scaled_list[n])

    cdf_list_ha=copy.deepcopy(cdf_list)
    cdf_scaled_list_ha=copy.deepcopy(cdf_scaled_list)

    HA_tcf_calc[SYS]['DiG']={}
    HA_tcf_calc[SYS]['DiG']['non_scaled']={}  #for two different barrier treatment
    HA_tcf_calc[SYS]['DiG']['scaled']={}
    HA_tcf_calc[SYS]['DiG']['CinvH']={}    #this is Qmatrix.npy file but it is actually (CH^-1) = (HC^-1)^-1 = (HA)^-1 (should've named it properly I'm stoopid lol)
    HA_tcf_calc[SYS]['DiG']['C']={}
    HA_tcf_calc[SYS]['DiG']['non_scaled']['tcf']=corr_func_list_ha
    HA_tcf_calc[SYS]['DiG']['non_scaled']['cdf']=cdf_list_ha
    HA_tcf_calc[SYS]['DiG']['scaled']['tcf']=corr_func_scaled_list_ha
    HA_tcf_calc[SYS]['DiG']['scaled']['cdf']=cdf_scaled_list_ha
    HA_tcf_calc[SYS]['DiG']['CinvH']['eigvec']=eigvecs_list
    HA_tcf_calc[SYS]['DiG']['CinvH']['eigval']=eigvals_list  #lambda eig vals coming from the HC^-1 
    HA_tcf_calc[SYS]['DiG']['C']['eigval']=C_eigvals_list    #mu eig values from Covariance matrix
    HA_tcf_calc[SYS]['DiG']['n_samples']=num_samples_per_msa
    HA_tcf_calc[SYS]['DiG']['t_axis']=t

/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
3ttp 99 320
fratio:  0.20111609925391974
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0


/tmp/ipykernel_2600544/699736766.py:43: RuntimeWarning: invalid value encountered in sqrt
  tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))


Time taken for 3ttp_0eafb is 0.01412mins
/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, without unitcells>
2pc0 99 320
fratio:  0.2031466528355126
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 2pc0_5d034 is 0.01526mins
/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 320 frames, 95 atoms, 95 residues, without unitcells>
<mdtraj.Trajectory with 320 frames, 95 atoms, 95 residues, without unitcells>
1q9p 95 320
fratio:  0.20384718294951396
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 1q9p_87f3d is 0.01285mins


In [12]:
suffix_list = ['_0eafb', '_5d034', '_87f3d']
for i,system_name in enumerate(['3ttp', '2pc0', '1q9p']):
    eigvals_list = []
    eigvecs_list = []
    C_eigvals_list = []
    tau_list = []
    tau_scaled_list = []
    corr_func_list = []
    corr_func_scaled_list = []
    cdf_list = []
    cdf_scaled_list = []
    covar_list = []
    dtraj_list = []
    SYS=system_name
    suffix = suffix_list[i]

    Hs=[]
    covars=[]
    AIHIs=[]
    start_time_all = time.time()
    
    path_structures=path_prefix + f'bioemu/{system_name}/'
    #os.chdir(path_structures)
    print(os.getcwd())
    XTC=path_structures + f'aligned.xtc'
    TOP=f"../{system_name}_CA.pdb"
    T=300
    NFRS = 320
    
    traj = traj_from_xtc(XTC, TOP)
    dtraj = traj - traj.mean(0)[None,:]
    QINV = np.load(path_structures + 'LE4PD/QINVmatrix.npy')
    Q = np.load(path_structures + 'LE4PD/Qmatrix.npy')
    eigvecs = np.copy(Q)
    eigvals = np.loadtxt(path_structures + 'LE4PD/lambda_eig')
    mu_eig = np.loadtxt(path_structures + 'LE4PD/mu_eig')
    eigvals_list.append(eigvals)
    eigvecs_list.append(Q)
    C_eigvals_list.append(mu_eig)
    
    fratio, sigma, fric, avfr = LE4PD3N.fric_calc(TOP, SYS, dtraj.shape[0] // 3, dtraj.shape[1], eigvals, 300, path_to_resarea=path_structures)
    tau = tau_convert(eigvals, avfr)
    tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))
    tau_list.append(tau)
    tau_scaled_list.append(tau_scaled)
    weights = np.zeros((eigvecs.shape[0] // 3, eigvecs.shape[1]))
    for n, _ in enumerate(range(0, eigvecs.shape[0], 3)):
        for a in range(eigvecs.shape[1]):
            #print(a, counter)
            weights[n,a] = (eigvecs[_:_+3,a]**2).sum(0)
    #t = np.linspace(0, 100000, 100000)
    import copy
    t = copy.deepcopy(t_axis) # known time axis values with 5ps gap
    corr_func = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau[None,:-6]))).T).T 
    corr_func_scaled = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau_scaled[None,:-6]))).T).T
    corr_func = corr_func / corr_func[0,:]
    corr_func_scaled = corr_func_scaled / corr_func_scaled[0,:]
    
    corr_func_list.append(corr_func)
    corr_func_scaled_list.append(corr_func_scaled)
    cdf_list.append(1. - corr_func)
    cdf_scaled_list.append(1. - corr_func_scaled)
    
    stop_time_all = time.time()
    
    print(f"Time taken for {system_name}{suffix} is {(stop_time_all-start_time_all)/60:2.4}mins")

    import copy
    corr_func_list_ha=[]
    corr_func_scaled_list_ha=[]
    corr_func_list_ha=copy.deepcopy(corr_func_list)
    corr_func_scaled_list_ha=copy.deepcopy(corr_func_scaled_list)
    cdf_list=[]
    cdf_scaled_list=[]
    for n, corr_func in enumerate(corr_func_list):
        cdf_list.append(1-corr_func)
        cdf_scaled_list.append(1-corr_func_scaled_list[n])

    cdf_list_ha=copy.deepcopy(cdf_list)
    cdf_scaled_list_ha=copy.deepcopy(cdf_scaled_list)

    HA_tcf_calc[SYS]['BioEMU']={}
    HA_tcf_calc[SYS]['BioEMU']['non_scaled']={}  #for two different barrier treatment
    HA_tcf_calc[SYS]['BioEMU']['scaled']={}
    HA_tcf_calc[SYS]['BioEMU']['CinvH']={}    #this is Qmatrix.npy file but it is actually (CH^-1) = (HC^-1)^-1 = (HA)^-1 (should've named it properly I'm stoopid lol)
    HA_tcf_calc[SYS]['BioEMU']['C']={}
    HA_tcf_calc[SYS]['BioEMU']['non_scaled']['tcf']=corr_func_list_ha
    HA_tcf_calc[SYS]['BioEMU']['non_scaled']['cdf']=cdf_list_ha
    HA_tcf_calc[SYS]['BioEMU']['scaled']['tcf']=corr_func_scaled_list_ha
    HA_tcf_calc[SYS]['BioEMU']['scaled']['cdf']=cdf_scaled_list_ha
    HA_tcf_calc[SYS]['BioEMU']['CinvH']['eigvec']=eigvecs_list
    HA_tcf_calc[SYS]['BioEMU']['CinvH']['eigval']=eigvals_list  #lambda eig vals coming from the HC^-1 
    HA_tcf_calc[SYS]['BioEMU']['C']['eigval']=C_eigvals_list    #mu eig values from Covariance matrix
    HA_tcf_calc[SYS]['BioEMU']['n_samples']=num_samples_per_msa
    HA_tcf_calc[SYS]['BioEMU']['t_axis']=t

/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 199 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 199 frames, 99 atoms, 99 residues, without unitcells>
3ttp 99 199
fratio:  0.20111609925391974
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0


/tmp/ipykernel_2600544/1644047615.py:43: RuntimeWarning: invalid value encountered in sqrt
  tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))


Time taken for 3ttp_0eafb is 0.01506mins
/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 217 frames, 99 atoms, 99 residues, without unitcells>
<mdtraj.Trajectory with 217 frames, 99 atoms, 99 residues, without unitcells>
2pc0 99 217
fratio:  0.2031466528355126
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 2pc0_5d034 is 0.01548mins
/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 191 frames, 95 atoms, 95 residues, without unitcells>
<mdtraj.Trajectory with 191 frames, 95 atoms, 95 residues, without unitcells>
1q9p 95 191
fratio:  0.20384718294951396
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 1q9p_87f3d is 0.01409mins


In [13]:
suffix_list = ['_0eafb', '_5d034', '_87f3d']
for i,system_name in enumerate(['3ttp', '2pc0', '1q9p']):
    eigvals_list = []
    eigvecs_list = []
    C_eigvals_list = []
    tau_list = []
    tau_scaled_list = []
    corr_func_list = []
    corr_func_scaled_list = []
    cdf_list = []
    cdf_scaled_list = []
    covar_list = []
    dtraj_list = []
    SYS=system_name
    suffix = suffix_list[i]

    Hs=[]
    covars=[]
    AIHIs=[]
    start_time_all = time.time()
    
    path_structures=path_prefix + f'{system_name}_MD/'
    #os.chdir(path_structures)
    print(os.getcwd())
    XTC=path_structures + f'aligned.xtc'
    TOP=f"../{system_name}_CA.pdb"
    T=300
    NFRS = 320
    
    traj = traj_from_xtc(XTC, TOP)
    dtraj = traj - traj.mean(0)[None,:]
    QINV = np.load(path_structures + 'QINVmatrix.npy')
    Q = np.load(path_structures + 'Qmatrix.npy')
    eigvecs = np.copy(Q)
    eigvals = np.loadtxt(path_structures + 'lambda_eig')
    mu_eig = np.loadtxt(path_structures + 'mu_eig')
    eigvals_list.append(eigvals)
    eigvecs_list.append(Q)
    C_eigvals_list.append(mu_eig)
    
    fratio, sigma, fric, avfr = LE4PD3N.fric_calc(TOP, SYS, dtraj.shape[0] // 3, dtraj.shape[1], eigvals, 300, path_to_resarea=path_structures)
    tau = tau_convert(eigvals, avfr)
    tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))
    tau_list.append(tau)
    tau_scaled_list.append(tau_scaled)
    weights = np.zeros((eigvecs.shape[0] // 3, eigvecs.shape[1]))
    for n, _ in enumerate(range(0, eigvecs.shape[0], 3)):
        for a in range(eigvecs.shape[1]):
            #print(a, counter)
            weights[n,a] = (eigvecs[_:_+3,a]**2).sum(0)
    #t = np.linspace(0, 100000, 100000)
    import copy
    t = copy.deepcopy(t_axis) # known time axis values with 5ps gap
    corr_func = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau[None,:-6]))).T).T 
    corr_func_scaled = np.matmul(weights[:,:-6], (mu_eig[:-6] * (np.exp(-t[:,None] / tau_scaled[None,:-6]))).T).T
    corr_func = corr_func / corr_func[0,:]
    corr_func_scaled = corr_func_scaled / corr_func_scaled[0,:]
    
    corr_func_list.append(corr_func)
    corr_func_scaled_list.append(corr_func_scaled)
    cdf_list.append(1. - corr_func)
    cdf_scaled_list.append(1. - corr_func_scaled)
    
    stop_time_all = time.time()
    
    print(f"Time taken for {system_name}{suffix} is {(stop_time_all-start_time_all)/60:2.4}mins")

    import copy
    corr_func_list_ha=[]
    corr_func_scaled_list_ha=[]
    corr_func_list_ha=copy.deepcopy(corr_func_list)
    corr_func_scaled_list_ha=copy.deepcopy(corr_func_scaled_list)
    cdf_list=[]
    cdf_scaled_list=[]
    for n, corr_func in enumerate(corr_func_list):
        cdf_list.append(1-corr_func)
        cdf_scaled_list.append(1-corr_func_scaled_list[n])

    cdf_list_ha=copy.deepcopy(cdf_list)
    cdf_scaled_list_ha=copy.deepcopy(cdf_scaled_list)

    HA_tcf_calc[SYS]['MD']={}
    HA_tcf_calc[SYS]['MD']['non_scaled']={}  #for two different barrier treatment
    HA_tcf_calc[SYS]['MD']['scaled']={}
    HA_tcf_calc[SYS]['MD']['CinvH']={}    #this is Qmatrix.npy file but it is actually (CH^-1) = (HC^-1)^-1 = (HA)^-1 (should've named it properly I'm stoopid lol)
    HA_tcf_calc[SYS]['MD']['C']={}
    HA_tcf_calc[SYS]['MD']['non_scaled']['tcf']=corr_func_list_ha
    HA_tcf_calc[SYS]['MD']['non_scaled']['cdf']=cdf_list_ha
    HA_tcf_calc[SYS]['MD']['scaled']['tcf']=corr_func_scaled_list_ha
    HA_tcf_calc[SYS]['MD']['scaled']['cdf']=cdf_scaled_list_ha
    HA_tcf_calc[SYS]['MD']['CinvH']['eigvec']=eigvecs_list
    HA_tcf_calc[SYS]['MD']['CinvH']['eigval']=eigvals_list  #lambda eig vals coming from the HC^-1 
    HA_tcf_calc[SYS]['MD']['C']['eigval']=C_eigvals_list    #mu eig values from Covariance matrix
    HA_tcf_calc[SYS]['MD']['n_samples']=num_samples_per_msa
    HA_tcf_calc[SYS]['MD']['t_axis']=t

/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, and unitcells>
<mdtraj.Trajectory with 320 frames, 99 atoms, 99 residues, and unitcells>
3ttp 99 320
fratio:  0.20111609925391974
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0


/tmp/ipykernel_2600544/969551278.py:43: RuntimeWarning: invalid value encountered in sqrt
  tau_scaled = tau * np.exp((eps * np.sqrt(mu_eig)))


Time taken for 3ttp_0eafb is 0.0156mins
/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 321 frames, 99 atoms, 99 residues, and unitcells>
<mdtraj.Trajectory with 321 frames, 99 atoms, 99 residues, and unitcells>
2pc0 99 321
fratio:  0.2031466528355126
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 2pc0_5d034 is 0.01499mins
/media/ebeyerle/seagate/af2-dynamics/notebooks
<mdtraj.Trajectory with 320 frames, 95 atoms, 95 residues, and unitcells>
<mdtraj.Trajectory with 320 frames, 95 atoms, 95 residues, and unitcells>
1q9p 95 320
fratio:  0.20384718294951396
Temperature (K):  300
Internal viscosity factor:  2.71828
Viscosity (Pa s):  0.001
fd20 0.0
Time taken for 1q9p_87f3d is 0.01403mins


In [14]:
import pickle
with open('./HA_tcf_calc_raw_template.pkl','wb') as f:
    pickle.dump(HA_tcf_calc,f)

# similarly for other methods 

such that `HA_tcf_calc[SYS]` has the following as keys `dict_keys(['AF2', 'test', 'BioEMU', 'DiG', 'AFc', 'MD'])`

- `'AF2'` for rMSA
- `'test'` for direct MD computing  These are given above
- `'BioEMU'` for BioEMU
- `'DiG'` for DiG method
- `'AFc'` for AF cluster
- `'MD'`  for uniformly sampled MD samples

Except for test every other methods contain the following keys as shown for AF2 method: `dict_keys(['non_scaled', 'scaled', 'CinvH', 'C', 'msas', 'n_samples', 't_axis'])`